In [48]:
import xlwings as xw
import pandas as pd
from dataclasses import dataclass
from typing import Optional
from enum import Enum

In [49]:
# Constantes para nomes de planilhas
PLANILHA_RELATOR = 'Relator - Coletivas Apropriação'
PLANILHA_CRITERIO = 'Critério IU 6'
PLANILHA_BANCADA = 'Bancada Impositiva - Consulta'
PLANILHA_BOTAO = 'Sheet1'

In [50]:
class Parecer(Enum):
    """
    Enum que representa os diferentes pareceres que podem ser atribuídos a uma emenda.
    """
    APROVACAO = 1
    APROVACAO_PARCIAL = 13 
    REJEICAO = 38 

In [51]:
class Decisao(Enum):
    """
    Enum que representa as diferentes decisões que podem ser tomadas para uma emenda.
    """
    APROVACAO = 1
    APROVACAO_PARCIAL = 2
    REJEICAO = 3

In [52]:
# Dicionário que mapeia cada parecer para a decisão correspondente
DECISAO_PARA_PARECER = {
    Decisao.APROVACAO: Parecer.APROVACAO,
    Decisao.APROVACAO_PARCIAL: Parecer.APROVACAO_PARCIAL,
    Decisao.REJEICAO: Parecer.REJEICAO
}

In [53]:
@dataclass
class Emenda:
    """
    Classe que representa uma emenda e suas informações relevantes.
    
    Attributes:
        sequencial (str): O código sequencial da emenda.
        uo (int): Unidade orçamentária da emenda.
        funcional (str): Função da emenda.
        atendimento_setorial (float): Valor de atendimento setorial da emenda.
        valor_solicitado (float): Valor solicitado para a emenda.
        soma_atendimento_setorial (float): Soma do atendimento setorial.
        soma_valor_solicitado (float): Soma dos valores solicitados.
        tem_parcela_impositiva (Optional[float]): Indica se a emenda tem parcela impositiva.
        id_uso (Optional[int]): ID de uso da emenda.
        decisao_parecer (Optional[Parecer]): Parecer da emenda.
        parecer_padrao (Optional[Decisao]): Decisão padrão da emenda.
    """
    sequencial: str
    uo: int
    funcional: str
    atendimento_setorial: float
    valor_solicitado: float
    soma_atendimento_setorial: float
    soma_valor_solicitado: float
    tem_parcela_impositiva: Optional[float] = None
    id_uso: Optional[int] = None
    decisao_parecer: Optional[Parecer] = None
    parecer_padrao: Optional[Decisao] = None

In [54]:
# Função para verificar se a emenda está na 'Bancada Impositiva - Consulta'
def checa_bancada_impositiva(emenda: str, df_bancada: pd.DataFrame) -> bool:
    """
    Verifica se uma emenda está presente na planilha 'Bancada Impositiva - Consulta'.
    
    Args:
        emenda (str): O código da emenda a ser verificada.
        df_bancada (pd.DataFrame): DataFrame contendo as informações da bancada impositiva.
    
    Returns:
        bool: True se a emenda estiver presente, False caso contrário.
    """
    return emenda in df_bancada['Emenda'].values

In [55]:
# Função para calcular a 'Decisão' de acordo com a lógica fornecida
def calcula_decisao(emenda: Emenda, df_bancada: pd.DataFrame) -> Decisao:
    """
    Calcula a decisão de uma emenda com base em regras de atendimento setorial e presença na bancada impositiva.
    Em resumo, a fórmula retornará:
    1 se a soma em 'Atendimento Setorial' for igual à soma em 'Atendimento Setorial' (e ambas não forem zero), ou seja, se valor solicitado for = aprovado, então emenda aprovada;
    2 se a soma em 'Atendimento Setorial' for diferente de 'Atendimento Setorial' (mas não zero), ou se a emenda for encontrado na planilha de impositivas 
      (ou seja, aprovado parcial se atendido for menor que solicitado ou, se valor atendido discricionária for zero, se a emenda tiver parcela impositiva)
    3 se a soma em 'Atendimento Setorial' for zero e A2 não for encontrado na outra planilha (ou seja, emenda rejeitada)
    
    Args:
        emenda (Emenda): Objeto Emenda contendo os dados da emenda.
        df_bancada (pd.DataFrame): DataFrame contendo as informações da bancada impositiva.
    
    Returns:
        Parecer: Enum indicando o parecer (APROVACAO, APROVACAO_PARCIAL, REJEICAO).
    """
    if emenda.soma_atendimento_setorial != 0:
        if emenda.soma_atendimento_setorial == emenda.soma_valor_solicitado:
            return Decisao.APROVACAO
        else:
            return Decisao.APROVACAO_PARCIAL
    else:
        if checa_bancada_impositiva(emenda.sequencial, df_bancada):
            return Decisao.APROVACAO_PARCIAL
        else:
            return Decisao.REJEICAO

In [56]:
def calcula_id_uso(emenda: Emenda, df_criterio: pd.DataFrame) -> int:
    """
    Calcula o ID de uso para uma emenda com base em critérios de UO e funcional presentes na tabela de critério.
    
    Args:
        emenda (Emenda): Objeto Emenda contendo os dados da emenda.
        df_criterio (pd.DataFrame): DataFrame contendo os critérios de uso.
    
    Returns:
        int: ID de uso (6 ou 0).
    """

    count_x = df_criterio['UO'].eq(emenda.uo).sum()
    last_4_of_y = emenda.funcional[-4:]
    count_y = df_criterio['Ação'].eq(last_4_of_y).sum()
    
    # Condicional para retornar 6 ou 0
    if count_x == 1 and count_y == 1:
        return 6
    else:
        return 0

In [57]:
def calcula_parecer(decisao_parecer: Decisao) -> Parecer:
    """
    Mapeia a decisão para um parecer correspondente.
    
    Args:
        decisao_parecer (Decisao): Enum indicando a Decisao.
    
    Returns:
        Decisao: Enum indicando o parecer correspondente a Decisao.
    """
    return DECISAO_PARA_PARECER[decisao_parecer]

In [58]:
def formata_dataframe(df):
    """
      Atualiza o dataframe original, mantendo apenas até a última linha com valor numérico.
      Remove a última linha de contabilização e possíveis linhas vazias ao final da tabela
      Preenche células mescladas (NaN) com o último valor válido
    Args:
        df (pd.DataFrame): DataFrame a ser formatado.
    
    Returns:
        pd.DataFrame: DataFrame formatado.
    """
    
    ultima_linha_numerica = df[df['Emenda'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notna()].index[-1]
    df = df.loc[:ultima_linha_numerica].copy()
    
    df = df.ffill()
    return df

In [59]:
def somas_por_emendas(df_relator):
    """
    Calcula as somas de atendimento setorial e valor solicitado para cada emenda.
    
    Args:
        df_relator (pd.DataFrame): DataFrame contendo os dados das emendas.
    
    Returns:
        pd.DataFrame: DataFrame atualizado com colunas de soma para atendimento setorial e valor solicitado.
    """
    
    somas_atendimento_setorial = df_relator.groupby('Emenda')['Atendimento Setorial'].sum()
    somas_valor_solicitado = df_relator.groupby('Emenda')['Valor Solicitado'].sum()

    # Adiciona essas somas como novas colunas ao dataframe original, usando o valor da emenda como referência
    df_relator = df_relator.merge(somas_atendimento_setorial.rename('Soma Atendimento Setorial'), 
                              left_on='Emenda', right_index=True, how='left')
    df_relator = df_relator.merge(somas_valor_solicitado.rename('Soma Valor Solicitado'), 
                              left_on='Emenda', right_index=True, how='left')
    return df_relator

In [60]:
def somas_parcelas_impositivas(df_bancada):
    """
    Calcula a soma dos valores solicitados para cada emenda na bancada impositiva.
    
    Args:
        df_bancada (pd.DataFrame): DataFrame contendo os dados da bancada impositiva.
    
    Returns:
        pd.DataFrame: DataFrame atualizado com a coluna de soma de valores solicitados.
    """
    somas_parcela_impositiva = df_bancada.groupby('Emenda')['Valor Solicitado'].sum()
    df_bancada = df_bancada.merge(somas_parcela_impositiva.rename('Soma Valor Solicitado'), 
                              left_on='Emenda', right_index=True, how='left')
    
    return df_bancada

In [61]:
def mapeia_parcelas_impositivas(df_bancada,df_relator):
    """
    Mapeia parcelas impositivas do DataFrame da bancada para o DataFrame do relator.
    
    Args:
        df_bancada (pd.DataFrame): DataFrame contendo os dados da bancada impositiva.
        df_relator (pd.DataFrame): DataFrame contendo os dados do relator.
    
    Returns:
        pd.DataFrame: DataFrame do relator atualizado com a coluna de parcelas impositivas.
    """
    # Criar um dicionário de mapeamento a partir do DataFrame df_bancada
    mapeamento = dict(zip(df_bancada['Emenda'], df_bancada['Soma Valor Solicitado']))

    # Atualizar a coluna 'Tem parcela impositiva?' do df_relator usando o método .map()
    # Caso não haja correspondência, preenche com '-'
    df_relator['Tem parcela impositiva?'] = df_relator['Emenda'].map(mapeamento).fillna('-')

    return df_relator

In [62]:
def processa_emenda(linha: pd.Series, df_criterio: pd.DataFrame, df_bancada: pd.DataFrame, impositiva: bool = False) -> pd.Series:
    """
    Processa uma linha do DataFrame, calculando ID de uso, decisão do parecer e parecer padrão.
    
    Args:
        row (pd.Series): Linha do DataFrame contendo os dados da emenda.
        df_criterio (pd.DataFrame): DataFrame contendo os critérios de uso.
        df_bancada (pd.DataFrame): DataFrame contendo os dados da bancada impositiva.
    
    Returns:
        pd.Series: Linha atualizada com as novas colunas calculadas.
    """
    emenda = Emenda(
        sequencial=linha['Emenda'],
        uo=int(linha['UO']),
        funcional=linha['Funcional'],
        atendimento_setorial=linha['Atendimento Setorial'],
        valor_solicitado=linha['Valor Solicitado'],
        soma_atendimento_setorial=linha['Soma Atendimento Setorial'],
        soma_valor_solicitado=linha['Soma Valor Solicitado'],
        tem_parcela_impositiva=linha['Tem parcela impositiva?']
    )
    
    emenda.id_uso = calcula_id_uso(emenda, df_criterio)
    
    if impositiva:
       emenda.decisao_parecer = Decisao.APROVACAO 
    else:
       emenda.decisao_parecer = calcula_decisao(emenda, df_bancada)
    
    emenda.parecer_padrao = calcula_parecer(emenda.decisao_parecer)
    
    linha['ID Uso'] = emenda.id_uso
    linha['Decisão Parecer'] = emenda.decisao_parecer.value
    linha['Parecer Padrão'] = emenda.parecer_padrao.value
    linha['Valor'] = linha['Atendimento Setorial']
    
    return linha

In [63]:
def processa_emendas(df_relator: pd.DataFrame, df_criterio: pd.DataFrame, df_bancada: pd.DataFrame, impositivas: bool = False) -> pd.DataFrame:
    """
    Processa todas as emendas no DataFrame do relator.
    
    Args:
        df_relator (pd.DataFrame): DataFrame contendo os dados das emendas do relator.
        df_criterio (pd.DataFrame): DataFrame contendo os critérios de uso.
        df_bancada (pd.DataFrame): DataFrame contendo os dados da bancada impositiva.
    
    Returns:
        pd.DataFrame: DataFrame do relator atualizado com as novas colunas calculadas.
    """
    return df_relator.apply(processa_emenda, axis=1, args=(df_criterio, df_bancada,impositivas))

In [64]:

    """
    Função principal para carregar os dados das planilhas, processar as emendas e atualizar as planilhas.
    """
    xw.Book("processaemendas.xlsm").set_mock_caller()
    # Conectar ao livro de trabalho ativo no xlwings
    wb = xw.Book.caller()  # Isso conecta ao arquivo Excel aberto pelo xlwings
    sheet_plansheet = wb.sheets[PLANILHA_BOTAO]
    sheet_plansheet.range('A1').value = 'Gerando planilha Lexor...'
  

    # Carregar as abas 
    sheet_apropriacao = wb.sheets[PLANILHA_RELATOR]
    sheet_criterio = wb.sheets[PLANILHA_CRITERIO]
    sheet_bancada = wb.sheets[PLANILHA_BANCADA]
    # Carregar as abas como DataFrames pandas
    df_relator =  sheet_apropriacao.used_range.options(pd.DataFrame, index=False, header=True).value
    df_criterio = sheet_criterio.used_range.options(pd.DataFrame, index=False, header=True).value
    df_bancada = sheet_bancada.used_range.options(pd.DataFrame, index=False, header=True).value
    

In [65]:
df_bancada

,Emenda,Emenda (Tipo),Autor,UO (Cod/Desc),Funcional (Sub),Ação + Subtítulo,GND,MA,RP,IU,...,Valor,UO,Funcional,Subtítulo Padrão ou Não Padrão Existente,Subtítulo Novo,Localidade,Justificativa,Decisão Parecer,Parecer Padrão,Texto Parecer Específico
0,71020009,INCLUSÃO - APROPRIAÇÃO,BANCADA DO ACRE,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0172,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,30,7,0,...,1000000.0,56101,15.451.2319.00T1,0172,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
1,71050003,INCLUSÃO - APROPRIAÇÃO,BANCADA DO AMAPA,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.7003,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",4,30,7,0,...,10000000.0,56101,15.451.5601.00SY,7003,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
2,71050004,INCLUSÃO - APROPRIAÇÃO,BANCADA DO AMAPA,56902 - FUNDO NACIONAL DE HABITAÇÃO DE INTERES...,16.482.2320.00TI.7001,APOIO À PRODUÇÃO HABITACIONAL DE INTERESSE SOC...,4,30,7,0,...,15624188.0,56902,16.482.2320.00TI,7001,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
3,71090007,INCLUSÃO - APROPRIAÇÃO,BANCADA DO ESPIRITO SANTO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.7012,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,30,7,0,...,25000000.0,56101,15.451.2319.00T1,7012,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
4,71100003,ACRÉSCIMO - APROPRIAÇÃO,BANCADA DE GOIAS,56202 - COMPANHIA BRASILEIRA DE TRENS URBANOS ...,15.453.0032.2843.0001,FUNCIONAMENTO DOS SISTEMAS DE TRANSPORTE FERRO...,4,90,7,0,...,8446651.0,56202,15.453.0032.2843,0001,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
5,71100014,INCLUSÃO - APROPRIAÇÃO,BANCADA DE GOIAS,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0052,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,3,90,7,0,...,10000000.0,56101,28.846.2320.00CW,0052,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
6,71120008,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.5401,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,40,7,0,...,28812094.0,56101,15.451.2319.00T1,5401,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
7,71120009,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.127.5602.00SW.5411,APOIO À REGULARIZAÇÃO FUNDIÁRIA URBANA - NO MU...,4,40,7,0,...,14000000.0,56101,15.127.5602.00SW,5411,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
8,71130010,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO DO SUL,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0054,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,3,90,7,0,...,35000000.0,56101,28.846.2320.00CW,0054,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
9,71160012,INCLUSÃO - APROPRIAÇÃO,BANCADA DA PARAIBA,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.1436,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,40,7,0,...,10300000.0,56101,15.451.2319.00T1,1436,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None


In [66]:
    # Formate dataframes
    df_bancada = formata_dataframe(df_bancada)
    df_relator = formata_dataframe(df_relator)

In [67]:
df_bancada

,Emenda,Emenda (Tipo),Autor,UO (Cod/Desc),Funcional (Sub),Ação + Subtítulo,GND,MA,RP,IU,...,Valor,UO,Funcional,Subtítulo Padrão ou Não Padrão Existente,Subtítulo Novo,Localidade,Justificativa,Decisão Parecer,Parecer Padrão,Texto Parecer Específico
0,71020009,INCLUSÃO - APROPRIAÇÃO,BANCADA DO ACRE,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0172,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,30,7,0,...,1000000.0,56101,15.451.2319.00T1,0172,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
1,71050003,INCLUSÃO - APROPRIAÇÃO,BANCADA DO AMAPA,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.7003,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",4,30,7,0,...,10000000.0,56101,15.451.5601.00SY,7003,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
2,71050004,INCLUSÃO - APROPRIAÇÃO,BANCADA DO AMAPA,56902 - FUNDO NACIONAL DE HABITAÇÃO DE INTERES...,16.482.2320.00TI.7001,APOIO À PRODUÇÃO HABITACIONAL DE INTERESSE SOC...,4,30,7,0,...,15624188.0,56902,16.482.2320.00TI,7001,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
3,71090007,INCLUSÃO - APROPRIAÇÃO,BANCADA DO ESPIRITO SANTO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.7012,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,30,7,0,...,25000000.0,56101,15.451.2319.00T1,7012,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
4,71100003,ACRÉSCIMO - APROPRIAÇÃO,BANCADA DE GOIAS,56202 - COMPANHIA BRASILEIRA DE TRENS URBANOS ...,15.453.0032.2843.0001,FUNCIONAMENTO DOS SISTEMAS DE TRANSPORTE FERRO...,4,90,7,0,...,8446651.0,56202,15.453.0032.2843,0001,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
5,71100014,INCLUSÃO - APROPRIAÇÃO,BANCADA DE GOIAS,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0052,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,3,90,7,0,...,10000000.0,56101,28.846.2320.00CW,0052,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
6,71120008,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.5401,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,40,7,0,...,28812094.0,56101,15.451.2319.00T1,5401,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
7,71120009,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.127.5602.00SW.5411,APOIO À REGULARIZAÇÃO FUNDIÁRIA URBANA - NO MU...,4,40,7,0,...,14000000.0,56101,15.127.5602.00SW,5411,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
8,71130010,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO DO SUL,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0054,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,3,90,7,0,...,35000000.0,56101,28.846.2320.00CW,0054,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None
9,71160012,INCLUSÃO - APROPRIAÇÃO,BANCADA DA PARAIBA,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.1436,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,40,7,0,...,10300000.0,56101,15.451.2319.00T1,1436,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None


In [68]:
    # Realiza calulos auxiliares
    df_bancada = somas_parcelas_impositivas(df_bancada)

In [69]:
df_bancada

,Emenda,Emenda (Tipo),Autor,UO (Cod/Desc),Funcional (Sub),Ação + Subtítulo,GND,MA,RP,IU,...,UO,Funcional,Subtítulo Padrão ou Não Padrão Existente,Subtítulo Novo,Localidade,Justificativa,Decisão Parecer,Parecer Padrão,Texto Parecer Específico,Soma Valor Solicitado
0,71020009,INCLUSÃO - APROPRIAÇÃO,BANCADA DO ACRE,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0172,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,30,7,0,...,56101,15.451.2319.00T1,0172,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,1000000.0
1,71050003,INCLUSÃO - APROPRIAÇÃO,BANCADA DO AMAPA,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.7003,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",4,30,7,0,...,56101,15.451.5601.00SY,7003,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,10000000.0
2,71050004,INCLUSÃO - APROPRIAÇÃO,BANCADA DO AMAPA,56902 - FUNDO NACIONAL DE HABITAÇÃO DE INTERES...,16.482.2320.00TI.7001,APOIO À PRODUÇÃO HABITACIONAL DE INTERESSE SOC...,4,30,7,0,...,56902,16.482.2320.00TI,7001,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,15624188.0
3,71090007,INCLUSÃO - APROPRIAÇÃO,BANCADA DO ESPIRITO SANTO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.7012,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,30,7,0,...,56101,15.451.2319.00T1,7012,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,25000000.0
4,71100003,ACRÉSCIMO - APROPRIAÇÃO,BANCADA DE GOIAS,56202 - COMPANHIA BRASILEIRA DE TRENS URBANOS ...,15.453.0032.2843.0001,FUNCIONAMENTO DOS SISTEMAS DE TRANSPORTE FERRO...,4,90,7,0,...,56202,15.453.0032.2843,0001,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,8446651.0
5,71100014,INCLUSÃO - APROPRIAÇÃO,BANCADA DE GOIAS,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0052,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,3,90,7,0,...,56101,28.846.2320.00CW,0052,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,10000000.0
6,71120008,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.5401,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,40,7,0,...,56101,15.451.2319.00T1,5401,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,28812094.0
7,71120009,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.127.5602.00SW.5411,APOIO À REGULARIZAÇÃO FUNDIÁRIA URBANA - NO MU...,4,40,7,0,...,56101,15.127.5602.00SW,5411,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,14000000.0
8,71130010,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO DO SUL,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0054,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,3,90,7,0,...,56101,28.846.2320.00CW,0054,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,35000000.0
9,71160012,INCLUSÃO - APROPRIAÇÃO,BANCADA DA PARAIBA,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.1436,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,40,7,0,...,56101,15.451.2319.00T1,1436,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,10300000.0


In [26]:
    #df_relator = somas_por_emendas(df_relator)
    #df_relator = mapeia_parcelas_impositivas(df_bancada,df_relator)
    # Processa todas as emendas 
    #df_relator = processa_emendas(df_relator, df_criterio, df_bancada)
    df_bancada = processa_emendas(df_relator, df_criterio, df_bancada, impositivas=True)
    

In [27]:
df_bancada

,Emenda,Emenda (Tipo),Autor,UO (Cod/Desc),Funcional (Sub),Ação + Subtítulo,Tem parcela impositiva?,GND,MA,RP,...,Funcional,Subtítulo Padrão ou Não Padrão Existente,Subtítulo Novo,Localidade,Justificativa,Decisão Parecer,Parecer Padrão,Texto Parecer Específico,Soma Atendimento Setorial,Soma Valor Solicitado
0,50030001,ACRÉSCIMO - APROPRIAÇÃO,COM. MEIO AMB DESENV SUSTENTÁVEL,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.511.2322.00VJ.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIA DE ...",-,4,40,8,...,17.511.2322.00VJ,0001,None,None,/ Atendimento de emendas coletivas de apropri...,1,1,None,0.0,2.002000e+08
1,50170003,INCLUSÃO - APROPRIAÇÃO,COM. FINANÇAS E TRIBUTAÇÃO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0001,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,-,3,90,8,...,28.846.2320.00CW,0001,None,None,/ Atendimento de emendas coletivas de apropri...,1,1,None,0.0,4.000000e+05
2,50230002,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0001,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,-,4,90,8,...,15.451.2319.00T1,0001,None,None,/ Atendimento de emendas coletivas de apropri...,1,1,None,0.0,1.065000e+09
3,50230003,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.512.2322.00TO.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIAS DE...",-,4,90,8,...,17.512.2322.00TO,0001,None,None,/ Atendimento de emendas coletivas de apropri...,1,1,None,0.0,5.000000e+04
4,50230004,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.845.2320.00CY.0001,TRANSFERÊNCIAS AO FUNDO DE DESENVOLVIMENTO SOC...,-,3,90,8,...,28.845.2320.00CY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,1,1,None,0.0,2.005000e+07
5,50310004,INCLUSÃO - APROPRIAÇÃO,COM. FISC FINANCEIRA E CONTROLE,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.512.2322.00TN.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIAS EM...",-,4,99,6,...,17.512.2322.00TN,0001,None,None,/ Atendimento de emendas coletivas de apropri...,1,1,None,0.0,0.000000e+00
6,60110001,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.0001,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",-,4,30,8,...,15.451.5601.00SY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,1,1,None,0.0,3.850000e+08
7,60110001,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.0001,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",-,4,40,8,...,15.451.5601.00SY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,1,1,None,0.0,3.850000e+08
8,60110002,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0001,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,-,4,30,8,...,15.451.2319.00T1,0001,None,None,/ Atendimento de emendas coletivas de apropri...,1,1,None,0.0,9.550000e+08
9,60110002,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0001,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,-,4,40,8,...,15.451.2319.00T1,0001,None,None,/ Atendimento de emendas coletivas de apropri...,1,1,None,0.0,9.550000e+08


In [ ]:
    
    # Escrever o DataFrame `df_relator de volta na planilha original
    sheet_apropriacao.range('A1').options(index=False).value = df_relator.iloc[:, :-2] # 'reset_index' evita o índice do Dataframe. Descartando as duas últimas colunas de soma criadas anteriormente
    # Escrever o DataFrame `df_relator de volta na planilha original
    sheet_bancada.range('A1').options(index=False).value = df_bancada.iloc[:,:-1] #Descartando a última coluna de soma de valor solicitado por emenda criada anteriormente


    sheet_plansheet.range('A1').value = 'Planilha Lexor gerada com sucesso!'

In [ ]:
if __name__ == "__main__":
    xw.Book("processaemendas.xlsm").set_mock_caller()
    main()